In [2]:
import boto3
import textwrap
import jsonlines
from collections import defaultdict

### General Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/General/Input/notes-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# load the service

client = boto3.client(service_name='comprehendmedical')

In [5]:
# make predictions

width = 20000
outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    texts = textwrap.wrap(note, width, 
                          break_long_words=False, break_on_hyphens=False, 
                          replace_whitespace=False, drop_whitespace=False)
    
    size = 0
    for text in texts:
        result = client.detect_phi(Text=text)
        for entity in result['Entities']:
            if entity['Type'] == 'NAME':
                outputs[ID][(size+entity['BeginOffset'], size+entity['EndOffset'])] = (entity['Text'], entity['Score'])
        size += len(text)
    
    if count % 1000 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 1000
Finish Processing Note 2000
Finish Processing Note 3000
Finish Processing Note 4000
Finish Processing Note 5000
Finish Processing Note 6000
Finish Processing Note 7000
Finish Processing Note 8000
Finish Processing Note 9000
Finish Processing Note 10000
Finish Processing Note 11000
Finish Processing Note 12000
Finish Processing Note 13000
Finish Processing Note 14000
Finish Processing Note 15000


In [7]:
# save the outputs

with jsonlines.open('../Data/General/Output/notes-Amazon.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])

### Polysemy Evaluation

In [2]:
# load the inputs

notes = {}
with jsonlines.open('../Data/Polysemy/Input/polysemies-input.jsonl', 'r') as reader:
    for line in reader:
        notes[tuple(line['ID'])] = line['note']

In [3]:
# install the AWS CLI, set up an IAM account with credentials, configure the AWS CLI, check the billing information
# load the service

client = boto3.client(service_name='comprehendmedical')

In [4]:
# make predictions

width = 20000
outputs = defaultdict(dict)
for count, (ID, note) in enumerate(notes.items()):
    texts = textwrap.wrap(note, width, 
                          break_long_words=False, break_on_hyphens=False, 
                          replace_whitespace=False, drop_whitespace=False)
    
    size = 0
    for text in texts:
        result = client.detect_phi(Text=text)
        for entity in result['Entities']:
            if entity['Type'] == 'NAME':
                outputs[ID][(size+entity['BeginOffset'], size+entity['EndOffset'])] = (entity['Text'], entity['Score'])
        size += len(text)
    
    if count % 100 == 0: print(f'Finish Processing Note {count}')

Finish Processing Note 0
Finish Processing Note 100
Finish Processing Note 200


In [5]:
# save the outputs

with jsonlines.open('../Data/Polysemy/Output/polysemies-Amazon.jsonl', 'w') as writer:
    writer.write_all([{'ID':list(ID), 'position':list(position), 'name':list(name)} for ID, preds in outputs.items() for position, name in preds.items()])